In [1]:
# start with the libraries
# get environment information

import sys
print(sys.version)
print(sys.executable)

3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 13:42:17) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
//anaconda3/envs/timeseries36/bin/python


In [2]:
# import the base libraries
# standard libraries
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import os
import math
from itertools import cycle
import datetime
from datetime import timedelta

# to visualize all of the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

import re
import csv


pd.options.display.max_colwidth = 2000
pd.set_option('display.max_rows', 100)


print(os.getcwd())

/Users/bryanbutler/Documents/Python/Forecasting/EWS


In [6]:
month_roll = pd.read_csv('wide_ews_fcast_02012020.csv', index_col='date_ride', parse_dates = True)
month_roll

,fcast_date,holiday,gross_rides,lower_gross_rides,upper_gross_rides,riders,lower_riders,upper_riders,cpm,lower_cpm,upper_cpm,vp_rides,lower_vp_rides,upper_vp_rides,vp_costs,lower_vp_costs,upper_vp_costs,vp_miles,lower_vp_miles,upper_vp_miles,vp_ambulatory_rides,lower_vp_ambulatory_rides,upper_vp_ambulatory_rides,vp_ambulatory_costs,lower_vp_ambulatory_costs,upper_vp_ambulatory_costs,vp_ambulatory_miles,lower_vp_ambulatory_miles,upper_vp_ambulatory_miles,vp_stretcher_rides,lower_vp_stretcher_rides,upper_vp_stretcher_rides,vp_stretcher_costs,lower_vp_stretcher_costs,upper_vp_stretcher_costs,vp_stretcher_miles,lower_vp_stretcher_miles,upper_vp_stretcher_miles,vp_wheelchair_rides,lower_vp_wheelchair_rides,upper_vp_wheelchair_rides,vp_wheelchair_costs,lower_vp_wheelchair_costs,upper_vp_wheelchair_costs,vp_wheelchair_miles,lower_vp_wheelchair_miles,upper_vp_wheelchair_miles
date_ride,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-03,2/1/20,0.00,nan,nan,nan,nan,nan,nan,2.50,2.36,2.63,"655,159.10","594,253.72","716,064.49","19,797,534.09","18,013,437.43","21,581,630.76","8,123,161.70","6,707,042.62","9,539,280.78","759,840.34","683,885.92","835,794.75","14,125,991.07","12,843,523.63","15,408,458.51","9,414,837.99","8,213,666.48","10,616,009.51","15,943.63","14,540.09","17,347.17","1,896,655.33","1,710,664.12","2,082,646.55","150,956.50","130,960.20","170,952.81","136,355.84","122,658.51","150,053.16","4,006,772.31","3,536,940.99","4,476,603.63","1,249,237.45","1,121,789.08","1,376,685.82"
2019-11-10,2/1/20,0.00,nan,nan,nan,nan,nan,nan,2.51,2.36,2.66,"635,445.26","573,828.42","697,062.11","19,731,905.81","17,784,149.31","21,679,662.30","7,949,169.74","6,448,547.90","9,449,791.58","766,801.10","677,216.70","856,385.50","13,718,985.74","12,423,578.34","15,014,393.15","9,498,228.37","8,114,557.65","10,881,899.09","15,950.54","14,338.83","17,562.26","1,872,287.60","1,671,602.67","2,072,972.53","151,151.04","128,145.72","174,156.36","138,040.48","121,876.90","154,204.05","4,030,237.40","3,514,690.00","4,545,784.79","1,262,031.28","1,114,626.95","1,409,435.61"
2019-11-17,2/1/20,0.00,nan,nan,nan,nan,nan,nan,2.49,2.33,2.66,"640,356.08","578,170.56","702,541.59","19,791,293.48","17,687,386.06","21,895,200.89","7,860,746.88","6,286,011.96","9,435,481.79","776,962.60","672,753.61","881,171.60","13,757,789.54","12,451,744.21","15,063,834.87","9,589,381.42","7,986,792.74","11,191,970.10","16,027.21","14,154.23","17,900.19","1,879,503.09","1,654,258.09","2,104,748.09","150,096.92","124,209.85","175,983.98","137,953.71","119,872.55","156,034.87","4,013,254.75","3,435,226.76","4,591,282.75","1,262,936.47","1,097,972.88","1,427,900.05"
2019-11-24,2/1/20,0.00,nan,nan,nan,nan,nan,nan,2.57,2.40,2.74,"508,410.52","445,769.47","571,051.56","16,011,092.87","13,785,325.92","18,236,859.81","6,458,316.17","4,813,732.32","8,102,900.01","673,557.34","547,330.89","799,783.78","11,037,159.62","9,722,354.20","12,351,965.05","8,194,403.80","6,314,594.64","10,074,212.96","14,486.11","12,332.22","16,640.00","1,653,242.52","1,408,254.81","1,898,230.24","126,769.61","98,223.63","155,315.58","127,107.98","107,176.45","147,039.51","3,450,257.13","2,820,672.66","4,079,841.59","1,101,955.28","921,129.59","1,282,780.98"
2019-12-01,2/1/20,1.00,nan,nan,nan,nan,nan,nan,2.50,2.32,2.68,"651,550.02","576,963.65","726,136.39","19,831,927.15","17,481,578.51","22,182,275.79","8,132,748.64","6,191,218.47","10,074,278.82","761,007.19","614,287.86","907,726.52","14,034,258.14","12,477,124.50","15,591,391.78","9,428,823.21","7,298,890.06","11,558,756.36","15,901.03","13,479.36","18,322.70","1,889,590.66","1,612,276.94","2,166,904.38","149,953.69","117,136.98","182,770.41","136,457.26","114,779.92","158,134.59","3,994,189.86","3,319,436.00","4,668,943.72","1,253,133.41","1,057,729.01","1,448,537.81"
2019-12-08,2/1/20,0.00,nan,nan,nan,nan,nan,nan,2.48,2.29,2.68,"656,503.58","580,969.78","732,037.39","19,794,556.88","17,299,397.01","22,289,716.75","8,227,445.82","6,149,787.13","10,30

In [7]:
# drop loast two rows

month_roll = month_roll[:-2].copy()
month_roll

,fcast_date,holiday,gross_rides,lower_gross_rides,upper_gross_rides,riders,lower_riders,upper_riders,cpm,lower_cpm,upper_cpm,vp_rides,lower_vp_rides,upper_vp_rides,vp_costs,lower_vp_costs,upper_vp_costs,vp_miles,lower_vp_miles,upper_vp_miles,vp_ambulatory_rides,lower_vp_ambulatory_rides,upper_vp_ambulatory_rides,vp_ambulatory_costs,lower_vp_ambulatory_costs,upper_vp_ambulatory_costs,vp_ambulatory_miles,lower_vp_ambulatory_miles,upper_vp_ambulatory_miles,vp_stretcher_rides,lower_vp_stretcher_rides,upper_vp_stretcher_rides,vp_stretcher_costs,lower_vp_stretcher_costs,upper_vp_stretcher_costs,vp_stretcher_miles,lower_vp_stretcher_miles,upper_vp_stretcher_miles,vp_wheelchair_rides,lower_vp_wheelchair_rides,upper_vp_wheelchair_rides,vp_wheelchair_costs,lower_vp_wheelchair_costs,upper_vp_wheelchair_costs,vp_wheelchair_miles,lower_vp_wheelchair_miles,upper_vp_wheelchair_miles
date_ride,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-03,2/1/20,0.00,nan,nan,nan,nan,nan,nan,2.50,2.36,2.63,"655,159.10","594,253.72","716,064.49","19,797,534.09","18,013,437.43","21,581,630.76","8,123,161.70","6,707,042.62","9,539,280.78","759,840.34","683,885.92","835,794.75","14,125,991.07","12,843,523.63","15,408,458.51","9,414,837.99","8,213,666.48","10,616,009.51","15,943.63","14,540.09","17,347.17","1,896,655.33","1,710,664.12","2,082,646.55","150,956.50","130,960.20","170,952.81","136,355.84","122,658.51","150,053.16","4,006,772.31","3,536,940.99","4,476,603.63","1,249,237.45","1,121,789.08","1,376,685.82"
2019-11-10,2/1/20,0.00,nan,nan,nan,nan,nan,nan,2.51,2.36,2.66,"635,445.26","573,828.42","697,062.11","19,731,905.81","17,784,149.31","21,679,662.30","7,949,169.74","6,448,547.90","9,449,791.58","766,801.10","677,216.70","856,385.50","13,718,985.74","12,423,578.34","15,014,393.15","9,498,228.37","8,114,557.65","10,881,899.09","15,950.54","14,338.83","17,562.26","1,872,287.60","1,671,602.67","2,072,972.53","151,151.04","128,145.72","174,156.36","138,040.48","121,876.90","154,204.05","4,030,237.40","3,514,690.00","4,545,784.79","1,262,031.28","1,114,626.95","1,409,435.61"
2019-11-17,2/1/20,0.00,nan,nan,nan,nan,nan,nan,2.49,2.33,2.66,"640,356.08","578,170.56","702,541.59","19,791,293.48","17,687,386.06","21,895,200.89","7,860,746.88","6,286,011.96","9,435,481.79","776,962.60","672,753.61","881,171.60","13,757,789.54","12,451,744.21","15,063,834.87","9,589,381.42","7,986,792.74","11,191,970.10","16,027.21","14,154.23","17,900.19","1,879,503.09","1,654,258.09","2,104,748.09","150,096.92","124,209.85","175,983.98","137,953.71","119,872.55","156,034.87","4,013,254.75","3,435,226.76","4,591,282.75","1,262,936.47","1,097,972.88","1,427,900.05"
2019-11-24,2/1/20,0.00,nan,nan,nan,nan,nan,nan,2.57,2.40,2.74,"508,410.52","445,769.47","571,051.56","16,011,092.87","13,785,325.92","18,236,859.81","6,458,316.17","4,813,732.32","8,102,900.01","673,557.34","547,330.89","799,783.78","11,037,159.62","9,722,354.20","12,351,965.05","8,194,403.80","6,314,594.64","10,074,212.96","14,486.11","12,332.22","16,640.00","1,653,242.52","1,408,254.81","1,898,230.24","126,769.61","98,223.63","155,315.58","127,107.98","107,176.45","147,039.51","3,450,257.13","2,820,672.66","4,079,841.59","1,101,955.28","921,129.59","1,282,780.98"
2019-12-01,2/1/20,1.00,nan,nan,nan,nan,nan,nan,2.50,2.32,2.68,"651,550.02","576,963.65","726,136.39","19,831,927.15","17,481,578.51","22,182,275.79","8,132,748.64","6,191,218.47","10,074,278.82","761,007.19","614,287.86","907,726.52","14,034,258.14","12,477,124.50","15,591,391.78","9,428,823.21","7,298,890.06","11,558,756.36","15,901.03","13,479.36","18,322.70","1,889,590.66","1,612,276.94","2,166,904.38","149,953.69","117,136.98","182,770.41","136,457.26","114,779.92","158,134.59","3,994,189.86","3,319,436.00","4,668,943.72","1,253,133.41","1,057,729.01","1,448,537.81"
2019-12-08,2/1/20,0.00,nan,nan,nan,nan,nan,nan,2.48,2.29,2.68,"656,503.58","580,969.78","732,037.39","19,794,556.88","17,299,397.01","22,289,716.75","8,227,445.82","6,149,787.13","10,30

In [8]:
# drop the confidence limits
regex = r"(upper_[a-z]*)|(lower_[a-z]*)"

month_roll = month_roll[month_roll.columns.drop(list(month_roll.filter(regex=regex)))]
month_roll.head()

,fcast_date,holiday,gross_rides,riders,cpm,vp_rides,vp_costs,vp_miles,vp_ambulatory_rides,vp_ambulatory_costs,vp_ambulatory_miles,vp_stretcher_rides,vp_stretcher_costs,vp_stretcher_miles,vp_wheelchair_rides,vp_wheelchair_costs,vp_wheelchair_miles
date_ride,,,,,,,,,,,,,,,,,
2019-11-03,2/1/20,0.00,nan,nan,2.50,"655,159.10","19,797,534.09","8,123,161.70","759,840.34","14,125,991.07","9,414,837.99","15,943.63","1,896,655.33","150,956.50","136,355.84","4,006,772.31","1,249,237.45"
2019-11-10,2/1/20,0.00,nan,nan,2.51,"635,445.26","19,731,905.81","7,949,169.74","766,801.10","13,718,985.74","9,498,228.37","15,950.54","1,872,287.60","151,151.04","138,040.48","4,030,237.40","1,262,031.28"
2019-11-17,2/1/20,0.00,nan,nan,2.49,"640,356.08","19,791,293.48","7,860,746.88","776,962.60","13,757,789.54","9,589,381.42","16,027.21","1,879,503.09","150,096.92","137,953.71","4,013,254.75","1,262,936.47"
2019-11-24,2/1/20,0.00,nan,nan,2.57,"508,410.52","16,011,092.87","6,458,316.17","673,557.34","11,037,159.62","8,194,403.80","14,486.11","1,653,242.52","126,769.61","127,107.98","3,450,257.13","1,101,955.28"
2019-12-01,2/1/20,1.00,nan,nan,2.50,"651,550.02","19,831,927.15","8,132,748.64","761,007.19","14,034,258.14","9,428,823.21","15,901.03","1,889,590.66","149,953.69","136,457.26","3,994,189.86","1,253,133.41"


In [9]:
month_roll.reset_index(inplace = True)
month_roll = month_roll.rename(columns={'date_ride': 'date'})
month_roll = month_roll.fillna(0)

month_roll

,date,fcast_date,holiday,gross_rides,riders,cpm,vp_rides,vp_costs,vp_miles,vp_ambulatory_rides,vp_ambulatory_costs,vp_ambulatory_miles,vp_stretcher_rides,vp_stretcher_costs,vp_stretcher_miles,vp_wheelchair_rides,vp_wheelchair_costs,vp_wheelchair_miles
0,2019-11-03,2/1/20,0.00,0.00,0.00,2.50,"655,159.10","19,797,534.09","8,123,161.70","759,840.34","14,125,991.07","9,414,837.99","15,943.63","1,896,655.33","150,956.50","136,355.84","4,006,772.31","1,249,237.45"
1,2019-11-10,2/1/20,0.00,0.00,0.00,2.51,"635,445.26","19,731,905.81","7,949,169.74","766,801.10","13,718,985.74","9,498,228.37","15,950.54","1,872,287.60","151,151.04","138,040.48","4,030,237.40","1,262,031.28"
2,2019-11-17,2/1/20,0.00,0.00,0.00,2.49,"640,356.08","19,791,293.48","7,860,746.88","776,962.60","13,757,789.54","9,589,381.42","16,027.21","1,879,503.09","150,096.92","137,953.71","4,013,254.75","1,262,936.47"
3,2019-11-24,2/1/20,0.00,0.00,0.00,2.57,"508,410.52","16,011,092.87","6,458,316.17","673,557.34","11,037,159.62","8,194,403.80","14,486.11","1,653,242.52","126,769.61","127,107.98","3,450,257.13","1,101,955.28"
4,2019-12-01,2/1/20,1.00,0.00,0.00,2.50,"651,550.02","19,831,927.15","8,132,748.64","761,007.19","14,034,258.14","9,428,823.21","15,901.03","1,889,590.66","149,953.69","136,457.26","3,994,189.86","1,253,133.41"
5,2019-12-08,2/1/20,0.00,0.00,0.00,2.48,"656,503.58","19,794,556.88","8,227,445.82","766,747.46","14,161,420.77","9,525,475.13","15,927.08","1,896,351.07","150,320.18","137,945.44","3,997,531.19","1,262,916.29"
6,2019-12-15,2/1/20,0.00,0.00,0.00,2.49,"635,470.55","19,876,761.44","8,028,463.87","776,071.25","13,736,895.77","9,610,458.69","15,984.59","1,883,236.85","149,951.25","138,260.57","3,997,845.42","1,266,354.29"
7,2019-12-22,2/1/20,0.00,0.00,0.00,2.56,"508,224.06","16,085,061.96","6,420,876.22","673,320.94","11,021,205.38","8,230,758.53","14,476.93","1,646,084.15","125,510.97","126,881.96","3,463,991.57","1,142,471.55"
8,2019-12-29,2/1/20,1.00,0.00,0.00,2.55,"510,190.98","16,163,284.20","6,580,405.41","665,963.33","11,094,317.26","8,155,543.37","14,367.31","1,664,873.84","126,543.77","125,792.76","3,440,971.14","1,132,126.82"
9,2020-01-05,2/1/20,1.00,"756,582.58","370,203.13",2.48,"652,868.50","19,873,009.05","8,237,814.53","767,831.09","14,077,206.40","9,565,175.95","15,891.50","1,898,686.53","149,506.54","138,316.12","4,044,557.31","1,275,045.54"


In [10]:
month_roll['date'] = pd.to_datetime(month_roll['date'])

month_roll['day'] = month_roll['date'].dt.day

month_roll

,date,fcast_date,holiday,gross_rides,riders,cpm,vp_rides,vp_costs,vp_miles,vp_ambulatory_rides,vp_ambulatory_costs,vp_ambulatory_miles,vp_stretcher_rides,vp_stretcher_costs,vp_stretcher_miles,vp_wheelchair_rides,vp_wheelchair_costs,vp_wheelchair_miles,day
0,2019-11-03,2/1/20,0.00,0.00,0.00,2.50,"655,159.10","19,797,534.09","8,123,161.70","759,840.34","14,125,991.07","9,414,837.99","15,943.63","1,896,655.33","150,956.50","136,355.84","4,006,772.31","1,249,237.45",3
1,2019-11-10,2/1/20,0.00,0.00,0.00,2.51,"635,445.26","19,731,905.81","7,949,169.74","766,801.10","13,718,985.74","9,498,228.37","15,950.54","1,872,287.60","151,151.04","138,040.48","4,030,237.40","1,262,031.28",10
2,2019-11-17,2/1/20,0.00,0.00,0.00,2.49,"640,356.08","19,791,293.48","7,860,746.88","776,962.60","13,757,789.54","9,589,381.42","16,027.21","1,879,503.09","150,096.92","137,953.71","4,013,254.75","1,262,936.47",17
3,2019-11-24,2/1/20,0.00,0.00,0.00,2.57,"508,410.52","16,011,092.87","6,458,316.17","673,557.34","11,037,159.62","8,194,403.80","14,486.11","1,653,242.52","126,769.61","127,107.98","3,450,257.13","1,101,955.28",24
4,2019-12-01,2/1/20,1.00,0.00,0.00,2.50,"651,550.02","19,831,927.15","8,132,748.64","761,007.19","14,034,258.14","9,428,823.21","15,901.03","1,889,590.66","149,953.69","136,457.26","3,994,189.86","1,253,133.41",1
5,2019-12-08,2/1/20,0.00,0.00,0.00,2.48,"656,503.58","19,794,556.88","8,227,445.82","766,747.46","14,161,420.77","9,525,475.13","15,927.08","1,896,351.07","150,320.18","137,945.44","3,997,531.19","1,262,916.29",8
6,2019-12-15,2/1/20,0.00,0.00,0.00,2.49,"635,470.55","19,876,761.44","8,028,463.87","776,071.25","13,736,895.77","9,610,458.69","15,984.59","1,883,236.85","149,951.25","138,260.57","3,997,845.42","1,266,354.29",15
7,2019-12-22,2/1/20,0.00,0.00,0.00,2.56,"508,224.06","16,085,061.96","6,420,876.22","673,320.94","11,021,205.38","8,230,758.53","14,476.93","1,646,084.15","125,510.97","126,881.96","3,463,991.57","1,142,471.55",22
8,2019-12-29,2/1/20,1.00,0.00,0.00,2.55,"510,190.98","16,163,284.20","6,580,405.41","665,963.33","11,094,317.26","8,155,543.37","14,367.31","1,664,873.84","126,543.77","125,792.76","3,440,971.14","1,132,126.82",29
9,2020-01-05,2/1/20,1.00,"756,582.58","370,203.13",2.48,"652,868.50","19,873,009.05","8,237,814.53","767,831.09","14,077,206.40","9,565,175.95","15,891.50","1,898,686.53","149,506.54","138,316.12","4,044,557.31","1,275,045.54",5


In [11]:
# function to roll back first of month
# from datetime import timedelta

def roll_back(row):
    
    if row['day'] == 1:
        return row['date'] - timedelta(days = 1)
    else:
        return row['date']

In [12]:
month_roll['date'] = month_roll.apply(roll_back, axis = 1)
month_roll

,date,fcast_date,holiday,gross_rides,riders,cpm,vp_rides,vp_costs,vp_miles,vp_ambulatory_rides,vp_ambulatory_costs,vp_ambulatory_miles,vp_stretcher_rides,vp_stretcher_costs,vp_stretcher_miles,vp_wheelchair_rides,vp_wheelchair_costs,vp_wheelchair_miles,day
0,2019-11-03,2/1/20,0.00,0.00,0.00,2.50,"655,159.10","19,797,534.09","8,123,161.70","759,840.34","14,125,991.07","9,414,837.99","15,943.63","1,896,655.33","150,956.50","136,355.84","4,006,772.31","1,249,237.45",3
1,2019-11-10,2/1/20,0.00,0.00,0.00,2.51,"635,445.26","19,731,905.81","7,949,169.74","766,801.10","13,718,985.74","9,498,228.37","15,950.54","1,872,287.60","151,151.04","138,040.48","4,030,237.40","1,262,031.28",10
2,2019-11-17,2/1/20,0.00,0.00,0.00,2.49,"640,356.08","19,791,293.48","7,860,746.88","776,962.60","13,757,789.54","9,589,381.42","16,027.21","1,879,503.09","150,096.92","137,953.71","4,013,254.75","1,262,936.47",17
3,2019-11-24,2/1/20,0.00,0.00,0.00,2.57,"508,410.52","16,011,092.87","6,458,316.17","673,557.34","11,037,159.62","8,194,403.80","14,486.11","1,653,242.52","126,769.61","127,107.98","3,450,257.13","1,101,955.28",24
4,2019-11-30,2/1/20,1.00,0.00,0.00,2.50,"651,550.02","19,831,927.15","8,132,748.64","761,007.19","14,034,258.14","9,428,823.21","15,901.03","1,889,590.66","149,953.69","136,457.26","3,994,189.86","1,253,133.41",1
5,2019-12-08,2/1/20,0.00,0.00,0.00,2.48,"656,503.58","19,794,556.88","8,227,445.82","766,747.46","14,161,420.77","9,525,475.13","15,927.08","1,896,351.07","150,320.18","137,945.44","3,997,531.19","1,262,916.29",8
6,2019-12-15,2/1/20,0.00,0.00,0.00,2.49,"635,470.55","19,876,761.44","8,028,463.87","776,071.25","13,736,895.77","9,610,458.69","15,984.59","1,883,236.85","149,951.25","138,260.57","3,997,845.42","1,266,354.29",15
7,2019-12-22,2/1/20,0.00,0.00,0.00,2.56,"508,224.06","16,085,061.96","6,420,876.22","673,320.94","11,021,205.38","8,230,758.53","14,476.93","1,646,084.15","125,510.97","126,881.96","3,463,991.57","1,142,471.55",22
8,2019-12-29,2/1/20,1.00,0.00,0.00,2.55,"510,190.98","16,163,284.20","6,580,405.41","665,963.33","11,094,317.26","8,155,543.37","14,367.31","1,664,873.84","126,543.77","125,792.76","3,440,971.14","1,132,126.82",29
9,2020-01-05,2/1/20,1.00,"756,582.58","370,203.13",2.48,"652,868.50","19,873,009.05","8,237,814.53","767,831.09","14,077,206.40","9,565,175.95","15,891.50","1,898,686.53","149,506.54","138,316.12","4,044,557.31","1,275,045.54",5


In [13]:
# get the difference between days

month_roll['days'] = month_roll['date'].diff() / np.timedelta64(1, 'D')

month_roll = month_roll.fillna(0)

month_roll

,date,fcast_date,holiday,gross_rides,riders,cpm,vp_rides,vp_costs,vp_miles,vp_ambulatory_rides,vp_ambulatory_costs,vp_ambulatory_miles,vp_stretcher_rides,vp_stretcher_costs,vp_stretcher_miles,vp_wheelchair_rides,vp_wheelchair_costs,vp_wheelchair_miles,day,days
0,2019-11-03,2/1/20,0.00,0.00,0.00,2.50,"655,159.10","19,797,534.09","8,123,161.70","759,840.34","14,125,991.07","9,414,837.99","15,943.63","1,896,655.33","150,956.50","136,355.84","4,006,772.31","1,249,237.45",3,0.00
1,2019-11-10,2/1/20,0.00,0.00,0.00,2.51,"635,445.26","19,731,905.81","7,949,169.74","766,801.10","13,718,985.74","9,498,228.37","15,950.54","1,872,287.60","151,151.04","138,040.48","4,030,237.40","1,262,031.28",10,7.00
2,2019-11-17,2/1/20,0.00,0.00,0.00,2.49,"640,356.08","19,791,293.48","7,860,746.88","776,962.60","13,757,789.54","9,589,381.42","16,027.21","1,879,503.09","150,096.92","137,953.71","4,013,254.75","1,262,936.47",17,7.00
3,2019-11-24,2/1/20,0.00,0.00,0.00,2.57,"508,410.52","16,011,092.87","6,458,316.17","673,557.34","11,037,159.62","8,194,403.80","14,486.11","1,653,242.52","126,769.61","127,107.98","3,450,257.13","1,101,955.28",24,7.00
4,2019-11-30,2/1/20,1.00,0.00,0.00,2.50,"651,550.02","19,831,927.15","8,132,748.64","761,007.19","14,034,258.14","9,428,823.21","15,901.03","1,889,590.66","149,953.69","136,457.26","3,994,189.86","1,253,133.41",1,6.00
5,2019-12-08,2/1/20,0.00,0.00,0.00,2.48,"656,503.58","19,794,556.88","8,227,445.82","766,747.46","14,161,420.77","9,525,475.13","15,927.08","1,896,351.07","150,320.18","137,945.44","3,997,531.19","1,262,916.29",8,8.00
6,2019-12-15,2/1/20,0.00,0.00,0.00,2.49,"635,470.55","19,876,761.44","8,028,463.87","776,071.25","13,736,895.77","9,610,458.69","15,984.59","1,883,236.85","149,951.25","138,260.57","3,997,845.42","1,266,354.29",15,7.00
7,2019-12-22,2/1/20,0.00,0.00,0.00,2.56,"508,224.06","16,085,061.96","6,420,876.22","673,320.94","11,021,205.38","8,230,758.53","14,476.93","1,646,084.15","125,510.97","126,881.96","3,463,991.57","1,142,471.55",22,7.00
8,2019-12-29,2/1/20,1.00,0.00,0.00,2.55,"510,190.98","16,163,284.20","6,580,405.41","665,963.33","11,094,317.26","8,155,543.37","14,367.31","1,664,873.84","126,543.77","125,792.76","3,440,971.14","1,132,126.82",29,7.00
9,2020-01-05,2/1/20,1.00,"756,582.58","370,203.13",2.48,"652,868.50","19,873,009.05","8,237,814.53","767,831.09","14,077,206.40","9,565,175.95","15,891.50","1,898,686.53","149,506.54","138,316.12","4,044,557.31","1,275,045.54",5,7.00


In [14]:
# set the index
# extract the adjusted month
# reset the actual days
month_roll['month'] = month_roll['date'].dt.month
month_roll['day'] = month_roll['date'].dt.day

# reset the index
month_roll.set_index('date', inplace=True)
month_roll

,fcast_date,holiday,gross_rides,riders,cpm,vp_rides,vp_costs,vp_miles,vp_ambulatory_rides,vp_ambulatory_costs,vp_ambulatory_miles,vp_stretcher_rides,vp_stretcher_costs,vp_stretcher_miles,vp_wheelchair_rides,vp_wheelchair_costs,vp_wheelchair_miles,day,days,month
date,,,,,,,,,,,,,,,,,,,,
2019-11-03,2/1/20,0.00,0.00,0.00,2.50,"655,159.10","19,797,534.09","8,123,161.70","759,840.34","14,125,991.07","9,414,837.99","15,943.63","1,896,655.33","150,956.50","136,355.84","4,006,772.31","1,249,237.45",3,0.00,11
2019-11-10,2/1/20,0.00,0.00,0.00,2.51,"635,445.26","19,731,905.81","7,949,169.74","766,801.10","13,718,985.74","9,498,228.37","15,950.54","1,872,287.60","151,151.04","138,040.48","4,030,237.40","1,262,031.28",10,7.00,11
2019-11-17,2/1/20,0.00,0.00,0.00,2.49,"640,356.08","19,791,293.48","7,860,746.88","776,962.60","13,757,789.54","9,589,381.42","16,027.21","1,879,503.09","150,096.92","137,953.71","4,013,254.75","1,262,936.47",17,7.00,11
2019-11-24,2/1/20,0.00,0.00,0.00,2.57,"508,410.52","16,011,092.87","6,458,316.17","673,557.34","11,037,159.62","8,194,403.80","14,486.11","1,653,242.52","126,769.61","127,107.98","3,450,257.13","1,101,955.28",24,7.00,11
2019-11-30,2/1/20,1.00,0.00,0.00,2.50,"651,550.02","19,831,927.15","8,132,748.64","761,007.19","14,034,258.14","9,428,823.21","15,901.03","1,889,590.66","149,953.69","136,457.26","3,994,189.86","1,253,133.41",30,6.00,11
2019-12-08,2/1/20,0.00,0.00,0.00,2.48,"656,503.58","19,794,556.88","8,227,445.82","766,747.46","14,161,420.77","9,525,475.13","15,927.08","1,896,351.07","150,320.18","137,945.44","3,997,531.19","1,262,916.29",8,8.00,12
2019-12-15,2/1/20,0.00,0.00,0.00,2.49,"635,470.55","19,876,761.44","8,028,463.87","776,071.25","13,736,895.77","9,610,458.69","15,984.59","1,883,236.85","149,951.25","138,260.57","3,997,845.42","1,266,354.29",15,7.00,12
2019-12-22,2/1/20,0.00,0.00,0.00,2.56,"508,224.06","16,085,061.96","6,420,876.22","673,320.94","11,021,205.38","8,230,758.53","14,476.93","1,646,084.15","125,510.97","126,881.96","3,463,991.57","1,142,471.55",22,7.00,12
2019-12-29,2/1/20,1.00,0.00,0.00,2.55,"510,190.98","16,163,284.20","6,580,405.41","665,963.33","11,094,317.26","8,155,543.37","14,367.31","1,664,873.84","126,543.77","125,792.76","3,440,971.14","1,132,126.82",29,7.00,12


In [23]:
# use the dictionary to assign a days_inmonth value
# create a dictionary of months for allocating
# function to get the number of days in month
def get_monthdays(month):
    month_len = {1:31, 2:29, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
    return month_len.get(month)

# apply the function
month_roll['days_inmonth'] = month_roll['month'].apply(get_monthdays)

month_roll

,fcast_date,holiday,gross_rides,riders,cpm,vp_rides,vp_costs,vp_miles,vp_ambulatory_rides,vp_ambulatory_costs,vp_ambulatory_miles,vp_stretcher_rides,vp_stretcher_costs,vp_stretcher_miles,vp_wheelchair_rides,vp_wheelchair_costs,vp_wheelchair_miles,day,days,month,days_inmonth
date,,,,,,,,,,,,,,,,,,,,,
2019-11-03,2/1/20,0.00,0.00,0.00,2.50,"655,159.10","19,797,534.09","8,123,161.70","759,840.34","14,125,991.07","9,414,837.99","15,943.63","1,896,655.33","150,956.50","136,355.84","4,006,772.31","1,249,237.45",3,0.00,11,30
2019-11-10,2/1/20,0.00,0.00,0.00,2.51,"635,445.26","19,731,905.81","7,949,169.74","766,801.10","13,718,985.74","9,498,228.37","15,950.54","1,872,287.60","151,151.04","138,040.48","4,030,237.40","1,262,031.28",10,7.00,11,30
2019-11-17,2/1/20,0.00,0.00,0.00,2.49,"640,356.08","19,791,293.48","7,860,746.88","776,962.60","13,757,789.54","9,589,381.42","16,027.21","1,879,503.09","150,096.92","137,953.71","4,013,254.75","1,262,936.47",17,7.00,11,30
2019-11-24,2/1/20,0.00,0.00,0.00,2.57,"508,410.52","16,011,092.87","6,458,316.17","673,557.34","11,037,159.62","8,194,403.80","14,486.11","1,653,242.52","126,769.61","127,107.98","3,450,257.13","1,101,955.28",24,7.00,11,30
2019-11-30,2/1/20,1.00,0.00,0.00,2.50,"651,550.02","19,831,927.15","8,132,748.64","761,007.19","14,034,258.14","9,428,823.21","15,901.03","1,889,590.66","149,953.69","136,457.26","3,994,189.86","1,253,133.41",30,6.00,11,30
2019-12-08,2/1/20,0.00,0.00,0.00,2.48,"656,503.58","19,794,556.88","8,227,445.82","766,747.46","14,161,420.77","9,525,475.13","15,927.08","1,896,351.07","150,320.18","137,945.44","3,997,531.19","1,262,916.29",8,8.00,12,31
2019-12-15,2/1/20,0.00,0.00,0.00,2.49,"635,470.55","19,876,761.44","8,028,463.87","776,071.25","13,736,895.77","9,610,458.69","15,984.59","1,883,236.85","149,951.25","138,260.57","3,997,845.42","1,266,354.29",15,7.00,12,31
2019-12-22,2/1/20,0.00,0.00,0.00,2.56,"508,224.06","16,085,061.96","6,420,876.22","673,320.94","11,021,205.38","8,230,758.53","14,476.93","1,646,084.15","125,510.97","126,881.96","3,463,991.57","1,142,471.55",22,7.00,12,31
2019-12-29,2/1/20,1.00,0.00,0.00,2.55,"510,190.98","16,163,284.20","6,580,405.41","665,963.33","11,094,317.26","8,155,543.37","14,367.31","1,664,873.84","126,543.77","125,792.76","3,440,971.14","1,132,126.82",29,7.00,12,31


In [24]:
# create simple boolean converter
# df.apply(lambda x: func(x['col1'],x['col2']),axis=1)

def convert_end(row):
    # find the rows that end the month
    # mark it with end
    if row:
        return 'end'
    else:
        return '0'
    
def convert_begin(row):
    # find the rows that begin the month
    # makr it with begin
    if row:
        return 'begin'
    else:
        return '0'


def get_max(row):
    # get the begin and end strings
    max_string = max(row['end'], row['begin'])
    return max_string

In [25]:
# make indicator for beginning and end of month
week_pos = month_roll.copy()

# shift back for end
# returns booleans
week_pos['end'] = week_pos['month'].shift(-1) != week_pos['month']

# shift forward for begin
# returns boolean
week_pos['begin'] = week_pos['month'].shift(1) != week_pos['month']

# convert booleans to words
week_pos['end'] = week_pos['end'].apply(convert_end)
week_pos['begin'] = week_pos['begin'].apply(convert_begin)

# set the first position to True
week_pos['position'] = week_pos.apply(get_max, axis = 1)
week_pos.drop(['end', 'begin'], inplace = True, axis = 1)

week_pos

,fcast_date,holiday,gross_rides,riders,cpm,vp_rides,vp_costs,vp_miles,vp_ambulatory_rides,vp_ambulatory_costs,vp_ambulatory_miles,vp_stretcher_rides,vp_stretcher_costs,vp_stretcher_miles,vp_wheelchair_rides,vp_wheelchair_costs,vp_wheelchair_miles,day,days,month,days_inmonth,position
date,,,,,,,,,,,,,,,,,,,,,,
2019-11-03,2/1/20,0.00,0.00,0.00,2.50,"655,159.10","19,797,534.09","8,123,161.70","759,840.34","14,125,991.07","9,414,837.99","15,943.63","1,896,655.33","150,956.50","136,355.84","4,006,772.31","1,249,237.45",3,0.00,11,30,begin
2019-11-10,2/1/20,0.00,0.00,0.00,2.51,"635,445.26","19,731,905.81","7,949,169.74","766,801.10","13,718,985.74","9,498,228.37","15,950.54","1,872,287.60","151,151.04","138,040.48","4,030,237.40","1,262,031.28",10,7.00,11,30,0
2019-11-17,2/1/20,0.00,0.00,0.00,2.49,"640,356.08","19,791,293.48","7,860,746.88","776,962.60","13,757,789.54","9,589,381.42","16,027.21","1,879,503.09","150,096.92","137,953.71","4,013,254.75","1,262,936.47",17,7.00,11,30,0
2019-11-24,2/1/20,0.00,0.00,0.00,2.57,"508,410.52","16,011,092.87","6,458,316.17","673,557.34","11,037,159.62","8,194,403.80","14,486.11","1,653,242.52","126,769.61","127,107.98","3,450,257.13","1,101,955.28",24,7.00,11,30,0
2019-11-30,2/1/20,1.00,0.00,0.00,2.50,"651,550.02","19,831,927.15","8,132,748.64","761,007.19","14,034,258.14","9,428,823.21","15,901.03","1,889,590.66","149,953.69","136,457.26","3,994,189.86","1,253,133.41",30,6.00,11,30,end
2019-12-08,2/1/20,0.00,0.00,0.00,2.48,"656,503.58","19,794,556.88","8,227,445.82","766,747.46","14,161,420.77","9,525,475.13","15,927.08","1,896,351.07","150,320.18","137,945.44","3,997,531.19","1,262,916.29",8,8.00,12,31,begin
2019-12-15,2/1/20,0.00,0.00,0.00,2.49,"635,470.55","19,876,761.44","8,028,463.87","776,071.25","13,736,895.77","9,610,458.69","15,984.59","1,883,236.85","149,951.25","138,260.57","3,997,845.42","1,266,354.29",15,7.00,12,31,0
2019-12-22,2/1/20,0.00,0.00,0.00,2.56,"508,224.06","16,085,061.96","6,420,876.22","673,320.94","11,021,205.38","8,230,758.53","14,476.93","1,646,084.15","125,510.97","126,881.96","3,463,991.57","1,142,471.55",22,7.00,12,31,0
2019-12-29,2/1/20,1.00,0.00,0.00,2.55,"510,190.98","16,163,284.20","6,580,405.41","665,963.33","11,094,317.26","8,155,543.37","14,367.31","1,664,873.84","126,543.77","125,792.76","3,440,971.14","1,132,126.82",29,7.00,12,31,end


In [42]:
# make adjustment factors

def adj_weeks(row):
    # adjust the beginning for partial weeks
    # uses a cumulative amount
    cum_sum = 5.3
    begin_portion = {1:.05, 2:.3, 3:1.3, 4:2.3, 5: 3.3, 6: 4.3, 7:5.3, 8:5.8}
    end_portion = {1: 1, 2:2, 3:3, 4:4, 5:5, 6:5.25}
    factor = 1
    if row['position'] == 'begin':
        factor = begin_portion.get(row['day'])
        factor = (factor + 0.5 * row['holiday'])/cum_sum
            
    
    # adjust the last week for partial periods too
    elif row['position'] == 'end':
        if row['day'] < row['days_inmonth']:
            day_delta = row['days_inmonth'] - row['day']
            factor = end_portion.get(day_delta)
            factor = 1 + (factor - .5 * row['holiday'])/cum_sum
    else:
        # set all others to 1
        factor = 1
    return factor

In [43]:
week_pos['adj_factor'] = week_pos.apply(adj_weeks, axis = 1)
week_pos

,fcast_date,holiday,gross_rides,riders,cpm,vp_rides,vp_costs,vp_miles,vp_ambulatory_rides,vp_ambulatory_costs,vp_ambulatory_miles,vp_stretcher_rides,vp_stretcher_costs,vp_stretcher_miles,vp_wheelchair_rides,vp_wheelchair_costs,vp_wheelchair_miles,day,days,month,days_inmonth,position,adj_factor
date,,,,,,,,,,,,,,,,,,,,,,,
2019-11-03,2/1/20,0.00,0.00,0.00,2.50,"655,159.10","19,797,534.09","8,123,161.70","759,840.34","14,125,991.07","9,414,837.99","15,943.63","1,896,655.33","150,956.50","136,355.84","4,006,772.31","1,249,237.45",3,0.00,11,30,begin,0.25
2019-11-10,2/1/20,0.00,0.00,0.00,2.51,"635,445.26","19,731,905.81","7,949,169.74","766,801.10","13,718,985.74","9,498,228.37","15,950.54","1,872,287.60","151,151.04","138,040.48","4,030,237.40","1,262,031.28",10,7.00,11,30,0,1.00
2019-11-17,2/1/20,0.00,0.00,0.00,2.49,"640,356.08","19,791,293.48","7,860,746.88","776,962.60","13,757,789.54","9,589,381.42","16,027.21","1,879,503.09","150,096.92","137,953.71","4,013,254.75","1,262,936.47",17,7.00,11,30,0,1.00
2019-11-24,2/1/20,0.00,0.00,0.00,2.57,"508,410.52","16,011,092.87","6,458,316.17","673,557.34","11,037,159.62","8,194,403.80","14,486.11","1,653,242.52","126,769.61","127,107.98","3,450,257.13","1,101,955.28",24,7.00,11,30,0,1.00
2019-11-30,2/1/20,1.00,0.00,0.00,2.50,"651,550.02","19,831,927.15","8,132,748.64","761,007.19","14,034,258.14","9,428,823.21","15,901.03","1,889,590.66","149,953.69","136,457.26","3,994,189.86","1,253,133.41",30,6.00,11,30,end,1.00
2019-12-08,2/1/20,0.00,0.00,0.00,2.48,"656,503.58","19,794,556.88","8,227,445.82","766,747.46","14,161,420.77","9,525,475.13","15,927.08","1,896,351.07","150,320.18","137,945.44","3,997,531.19","1,262,916.29",8,8.00,12,31,begin,1.09
2019-12-15,2/1/20,0.00,0.00,0.00,2.49,"635,470.55","19,876,761.44","8,028,463.87","776,071.25","13,736,895.77","9,610,458.69","15,984.59","1,883,236.85","149,951.25","138,260.57","3,997,845.42","1,266,354.29",15,7.00,12,31,0,1.00
2019-12-22,2/1/20,0.00,0.00,0.00,2.56,"508,224.06","16,085,061.96","6,420,876.22","673,320.94","11,021,205.38","8,230,758.53","14,476.93","1,646,084.15","125,510.97","126,881.96","3,463,991.57","1,142,471.55",22,7.00,12,31,0,1.00
2019-12-29,2/1/20,1.00,0.00,0.00,2.55,"510,190.98","16,163,284.20","6,580,405.41","665,963.33","11,094,317.26","8,155,543.37","14,367.31","1,664,873.84","126,543.77","125,792.76","3,440,971.14","1,132,126.82",29,7.00,12,31,end,1.28


In [44]:
col_names = list(week_pos.columns)
col_names

['fcast_date',
 'holiday',
 'gross_rides',
 'riders',
 'cpm',
 'vp_rides',
 'vp_costs',
 'vp_miles',
 'vp_ambulatory_rides',
 'vp_ambulatory_costs',
 'vp_ambulatory_miles',
 'vp_stretcher_rides',
 'vp_stretcher_costs',
 'vp_stretcher_miles',
 'vp_wheelchair_rides',
 'vp_wheelchair_costs',
 'vp_wheelchair_miles',
 'day',
 'days',
 'month',
 'days_inmonth',
 'position',
 'adj_factor']

In [45]:
len(col_names)

23

In [46]:
adj_cpm = month_roll[['cpm']]

adj_cpm = adj_cpm.rename(columns={'cpm': 'cpm_fcast'})

adj_cpm

,cpm_fcast
date,
2019-11-03,2.50
2019-11-10,2.51
2019-11-17,2.49
2019-11-24,2.57
2019-11-30,2.50
2019-12-08,2.48
2019-12-15,2.49
2019-12-22,2.56
2019-12-29,2.55


In [47]:
col_names

['fcast_date',
 'holiday',
 'gross_rides',
 'riders',
 'cpm',
 'vp_rides',
 'vp_costs',
 'vp_miles',
 'vp_ambulatory_rides',
 'vp_ambulatory_costs',
 'vp_ambulatory_miles',
 'vp_stretcher_rides',
 'vp_stretcher_costs',
 'vp_stretcher_miles',
 'vp_wheelchair_rides',
 'vp_wheelchair_costs',
 'vp_wheelchair_miles',
 'day',
 'days',
 'month',
 'days_inmonth',
 'position',
 'adj_factor']

In [49]:
short_list = col_names[2:16]
short_list.remove('cpm')
short_list


['gross_rides',
 'riders',
 'vp_rides',
 'vp_costs',
 'vp_miles',
 'vp_ambulatory_rides',
 'vp_ambulatory_costs',
 'vp_ambulatory_miles',
 'vp_stretcher_rides',
 'vp_stretcher_costs',
 'vp_stretcher_miles',
 'vp_wheelchair_rides',
 'vp_wheelchair_costs']

In [50]:
# new_preds = week_pos[[adj_factor]]
# this works
# new_preds = week_pos[["gross_rides_fcast", "riders_fcast"]].multiply(week_pos["adj_factor"], axis="index")

new_preds = week_pos[short_list].multiply(week_pos["adj_factor"], axis="index")
new_preds

,gross_rides,riders,vp_rides,vp_costs,vp_miles,vp_ambulatory_rides,vp_ambulatory_costs,vp_ambulatory_miles,vp_stretcher_rides,vp_stretcher_costs,vp_stretcher_miles,vp_wheelchair_rides,vp_wheelchair_costs
date,,,,,,,,,,,,,
2019-11-03,0.00,0.00,"160,699.40","4,855,998.93","1,992,473.62","186,375.93","3,464,865.73","2,309,299.89","3,910.70","465,217.35","37,027.07","33,445.77","982,793.21"
2019-11-10,0.00,0.00,"635,445.26","19,731,905.81","7,949,169.74","766,801.10","13,718,985.74","9,498,228.37","15,950.54","1,872,287.60","151,151.04","138,040.48","4,030,237.40"
2019-11-17,0.00,0.00,"640,356.08","19,791,293.48","7,860,746.88","776,962.60","13,757,789.54","9,589,381.42","16,027.21","1,879,503.09","150,096.92","137,953.71","4,013,254.75"
2019-11-24,0.00,0.00,"508,410.52","16,011,092.87","6,458,316.17","673,557.34","11,037,159.62","8,194,403.80","14,486.11","1,653,242.52","126,769.61","127,107.98","3,450,257.13"
2019-11-30,0.00,0.00,"651,550.02","19,831,927.15","8,132,748.64","761,007.19","14,034,258.14","9,428,823.21","15,901.03","1,889,590.66","149,953.69","136,457.26","3,994,189.86"
2019-12-08,0.00,0.00,"718,437.89","21,661,967.91","9,003,619.96","839,082.13","15,497,403.86","10,424,104.86","17,429.63","2,075,252.11","164,501.33","150,959.16","4,374,656.77"
2019-12-15,0.00,0.00,"635,470.55","19,876,761.44","8,028,463.87","776,071.25","13,736,895.77","9,610,458.69","15,984.59","1,883,236.85","149,951.25","138,260.57","3,997,845.42"
2019-12-22,0.00,0.00,"508,224.06","16,085,061.96","6,420,876.22","673,320.94","11,021,205.38","8,230,758.53","14,476.93","1,646,084.15","125,510.97","126,881.96","3,463,991.57"
2019-12-29,0.00,0.00,"654,584.66","20,737,798.60","8,442,784.29","854,443.52","14,234,218.37","10,463,716.02","18,433.53","2,136,064.55","162,358.05","161,394.48","4,414,830.89"


In [51]:
# extract the month after adjustments

month_preds = new_preds.resample('M').aggregate({np.sum})

month_preds

,gross_rides,riders,vp_rides,vp_costs,vp_miles,vp_ambulatory_rides,vp_ambulatory_costs,vp_ambulatory_miles,vp_stretcher_rides,vp_stretcher_costs,vp_stretcher_miles,vp_wheelchair_rides,vp_wheelchair_costs
,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum
date,,,,,,,,,,,,,
2019-11-30,0.00,0.00,"2,596,461.28","80,222,218.24","32,393,455.05","3,164,704.16","56,013,058.77","39,020,136.69","66,275.59","7,759,841.22","614,998.32","573,005.19","16,470,732.34"
2019-12-31,0.00,0.00,"2,516,717.15","78,361,589.90","31,895,744.34","3,142,917.84","54,489,723.38","38,729,038.09","66,324.68","7,740,637.67","602,321.59","577,496.17","16,251,324.66"
2020-01-31,"3,757,820.72","1,835,811.39","2,815,018.00","87,320,280.43","35,685,156.21","3,413,621.17","60,795,575.50","42,353,395.85","71,151.90","8,419,244.04","657,776.56","618,945.86","17,720,953.49"
2020-02-29,"3,500,059.15","1,709,245.25","2,491,593.15","77,475,384.17","31,979,236.16","3,033,651.45","53,865,400.43","37,745,233.54","62,982.06","7,496,259.12","581,370.86","552,001.57","15,652,340.01"


In [ ]:
adj_cpm = adj_cpm.resample('M').aggregate({np.mean})

adj_cpm